# Computational graph example with PyTorch

In [1]:
!pip install torch

In [1]:
import torch


In [5]:
z = None
z.sort()

AttributeError: 'NoneType' object has no attribute 'sort'

In [10]:
import torch
import torch.nn as nn

# Automatischen Gradienten aktivieren (Standard in PyTorch)
torch.set_grad_enabled(True)

print("=" * 60)
print("EINFACHES BERECHNUNGSGRAPH-BEISPIEL IN PYTORCH")
print("=" * 60)

# 1. Einfaches Beispiel: Grundlegende Operationen
print("\n1. EINFACHES BEISPIEL:")
print("-" * 40)

# Eingabetensoren mit requires_grad=True für Gradientenberechnung
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Vorwärtsberechnung (Forward Pass)
# PyTorch baut automatisch den Berechnungsgraphen auf
z = x * y  # z = 2 * 3 = 6
z.retain_grad()
w = z + x  # w = 6 + 2 = 8
w.retain_grad()
loss = w ** 2  # loss = 8^2 = 64

print(f"x = {x.item():.2f}")
print(f"y = {y.item():.2f}")
print(f"z = x * y = {z.item():.2f}")
print(f"w = z + x = {w.item():.2f}")
print(f"loss = w² = {loss.item():.2f}")

# Rückwärtsberechnung (Backward Pass)
loss.backward()

print(f"\nGradienten nach backward():")
print(f"∂loss/∂x = {x.grad.item():.2f}")
print(f"∂loss/∂y = {y.grad.item():.2f}")

print("Gradienten der Zwischentensoren:")
print(f"∂loss/∂z = {z.grad.item():.2f}")
print(f"∂loss/∂w = {w.grad.item():.2f}")

# Manuelle Überprüfung der Gradienten:
# f: R^2 --> R, (x,y) |-> loss
# loss = (x*y + x)² = (2*3 + 2)² = 8² = 64
# ∂loss/∂x = 2*(x*y + x) * (y + 1) = 2*8*4 = 64
# ∂loss/∂y = 2*(x*y + x) * x = 2*8*2 = 32



EINFACHES BERECHNUNGSGRAPH-BEISPIEL IN PYTORCH

1. EINFACHES BEISPIEL:
----------------------------------------
x = 2.00
y = 3.00
z = x * y = 6.00
w = z + x = 8.00
loss = w² = 64.00

Gradienten nach backward():
∂loss/∂x = 64.00
∂loss/∂y = 32.00
Gradienten der Zwischentensoren:
∂loss/∂z = 16.00
∂loss/∂w = 16.00


In [14]:
print("\n" + "=" * 60)
print("2. NEURONALES NETZWERK BEISPIEL:")
print("-" * 40)

# Gradienten zurücksetzen
x.grad = None
y.grad = None

# Ein einfaches neuronales Netzwerk definieren: 2->3->1 MLP
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 3)  # 2 Eingänge, 3 Neuronen
        self.fc2 = nn.Linear(3, 1)  # 3 Eingänge, 1 Ausgang
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Netzwerk erstellen
model = SimpleNet()

# Eingabedaten (Batch-Größe 1, 2 Features)
input_data = torch.tensor([[1.0, 2.0]], requires_grad=True)
target = torch.tensor([[5.0]])

# Forward Pass
output = model(input_data)
print(f"Eingabe: {input_data.data}")
print(f"Netzwerk-Ausgabe: {output.item():.4f}")
print(f"Zielwert: {target.item():.4f}")

# Verlustfunktion
criterion = nn.MSELoss()
loss = criterion(output, target)
print(f"Verlust (MSE): {loss.item():.4f}")

# Backward Pass
loss.backward()

print("\nGradienten der Netzwerk-Parameter:")
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name}: Gradient-Shape = {param.grad.shape}")
        print(f"  Gradient-Werte (erste 3): {param.grad.flatten().tolist()}")



2. NEURONALES NETZWERK BEISPIEL:
----------------------------------------
Eingabe: tensor([[1., 2.]])
Netzwerk-Ausgabe: 0.4904
Zielwert: 5.0000
Verlust (MSE): 20.3368

Gradienten der Netzwerk-Parameter:
fc1.weight: Gradient-Shape = torch.Size([3, 2])
  Gradient-Werte (erste 3): [0.0, 0.0, 4.784209728240967, 9.568419456481934, -4.932976722717285, -9.86595344543457]
fc1.bias: Gradient-Shape = torch.Size([3])
  Gradient-Werte (erste 3): [0.0, 4.784209728240967, -4.932976722717285]
fc2.weight: Gradient-Shape = torch.Size([1, 3])
  Gradient-Werte (erste 3): [0.0, -7.217965602874756, -13.513440132141113]
fc2.bias: Gradient-Shape = torch.Size([1])
  Gradient-Werte (erste 3): [-9.019274711608887]


In [12]:
(5-0.1101)**2

23.91112201

In [15]:
print("\n" + "=" * 60)
print("3. VISUALISIERUNG DES BERECHNUNGSGRAPHEN:")
print("-" * 40)

# Neues einfaches Beispiel für bessere Visualisierung
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a + b  # c = 5
d = a * b  # d = 6
e = c * d  # e = 30

print("Berechnungsgraph-Struktur:")
print("       a(2.0)   b(3.0)")
print("         |   \\ /   |")
print("         |    X    |")
print("         |   / \\   |")
print("       c=a+b    d=a*b")
print("        (5)      (6)")
print("         \\      /")
print("          \\    /")
print("           e=c*d")
print("           (30)")

e.backward()
print(f"\nWerte:")
print(f"a = {a.item():.1f}, b = {b.item():.1f}")
print(f"c = a + b = {c.item():.1f}")
print(f"d = a * b = {d.item():.1f}")
print(f"e = c * d = {e.item():.1f}")

print(f"\nGradienten:")
print(f"∂e/∂a = {a.grad.item():.1f}")  # ∂e/∂a = d + c = 6 + 5 = 11
print(f"∂e/∂b = {b.grad.item():.1f}")  # ∂e/∂b = d + c = 6 + 5 = 11

print("\n" + "=" * 60)
print("WICHTIGE KONZEPTE:")
print("-" * 40)
print("1. requires_grad=True aktiviert die Gradientenverfolgung")
print("2. backward() berechnet alle Gradienten automatisch")
print("3. PyTorch erstellt den Berechnungsgraphen dynamisch")
print("4. Gradienten werden akkumuliert (addiert)")
print("5. Mit .grad kann auf die Gradienten zugegriffen werden")
print("=" * 60)


3. VISUALISIERUNG DES BERECHNUNGSGRAPHEN:
----------------------------------------
Berechnungsgraph-Struktur:
       a(2.0)   b(3.0)
         |   \ /   |
         |    X    |
         |   / \   |
       c=a+b    d=a*b
        (5)      (6)
         \      /
          \    /
           e=c*d
           (30)

Werte:
a = 2.0, b = 3.0
c = a + b = 5.0
d = a * b = 6.0
e = c * d = 30.0

Gradienten:
∂e/∂a = 21.0
∂e/∂b = 16.0

WICHTIGE KONZEPTE:
----------------------------------------
1. requires_grad=True aktiviert die Gradientenverfolgung
2. backward() berechnet alle Gradienten automatisch
3. PyTorch erstellt den Berechnungsgraphen dynamisch
4. Gradienten werden akkumuliert (addiert)
5. Mit .grad kann auf die Gradienten zugegriffen werden
